In [1]:
import numpy as np 
import time

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import RobustScaler

from utils.inference import Trainer, plot_loss
import utils.datasets as d

In [42]:
import pandas as pd
data = pd.read_csv("data/Original_features.csv").dropna()

In [82]:
y = data.labels.values

In [83]:
X = data.drop(columns=["Unnamed: 0","labels"]).values

In [45]:
print(X.shape)
print(y.shape)

(83998, 29)
(83998,)


In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [89]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [48]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [49]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_validation, y_validation)
test = d.H_alphaSequences(X_test, y_test)

In [50]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [51]:
vars(scaler)

{'with_centering': True,
 'with_scaling': True,
 'quantile_range': (25.0, 75.0),
 'copy': True,
 'center_': array([ 7.57052205e-01, -9.75971436e-04,  4.62379578e-04,  7.59820060e-01,
         7.62030538e-01,  7.62700924e-01,  7.63305767e-01,  7.62163191e-01,
         7.61376262e-01,  7.61347926e-01,  6.53664985e-01,  5.95234821e-01,
         5.43589543e-01,  4.97168308e-01,  4.55740570e-01,  4.19610353e-01,
         3.87514742e-01,  1.47109883e-03,  1.71995368e-03,  1.93611755e-03,
         2.12938344e-03,  2.33138068e-03,  2.50169730e-03,  2.67065562e-03,
         2.84640982e-03,  3.02459783e-03,  3.18005887e-03,  3.32942749e-03,
         3.48481734e-03]),
 'scale_': array([0.57630414, 0.01369458, 0.00919271, 0.56743685, 0.56287884,
        0.55910696, 0.55559377, 0.55362712, 0.55214722, 0.55014138,
        0.48773622, 0.44029612, 0.39966551, 0.3640042 , 0.33393052,
        0.30716095, 0.28320795, 0.00348108, 0.00401186, 0.00449096,
        0.00495415, 0.00537281, 0.00578786, 0.006203

In [92]:
model = nn.Sequential(
            nn.Linear(in_features=29, out_features=64),
            nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=64),
            nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=64),
            nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=4)
)

In [93]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)

In [94]:
m1 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="NN_3x64_BN__Drop=3_sigmoid_lr=1e-3_bs=512",
        verbose=True
        )

cuda:0


In [95]:
lh = m1(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.7340, validation_loss:0.5455, val_accuracy:0.8047
Epoch [2/100], average_loss:0.5330, validation_loss:0.4750, val_accuracy:0.7891
Epoch [3/100], average_loss:0.4759, validation_loss:0.4492, val_accuracy:0.8047
Epoch [4/100], average_loss:0.4470, validation_loss:0.4105, val_accuracy:0.8516
Epoch [5/100], average_loss:0.4263, validation_loss:0.4163, val_accuracy:0.8438
Epoch [6/100], average_loss:0.4137, validation_loss:0.3999, val_accuracy:0.8672
Epoch [7/100], average_loss:0.4059, validation_loss:0.4238, val_accuracy:0.7969
Epoch [8/100], average_loss:0.4002, validation_loss:0.4242, val_accuracy:0.8203
Epoch [9/100], average_loss:0.3955, validation_loss:0.3709, val_accuracy:0.8125
Epoch [10/100], average_loss:0.3909, validation_loss:0.3749, val_accuracy:0.8047
Epoch [11/100], average_loss:0.3924, validation_loss:0.3842, val_accuracy:0.8594
Epoch [12/100], average_loss:0.3887, validation_loss:0.3917, val_accuracy:0.8281
Epoch [13/100], average_loss:0.3873, 

In [101]:
m1.tb.close()

In [96]:
m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [97]:
x_pred

tensor([1, 0, 1,  ..., 1, 0, 1])

In [98]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.792632182988572

In [99]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8507738095238095

In [100]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[2754, 1105,   59,  237],
       [ 407, 9867,   74,  249],
       [   9,   54,  408,   21],
       [ 181,   81,   30, 1264]], dtype=int64)

# wider

In [102]:
model = nn.Sequential(
            nn.Linear(in_features=29, out_features=128),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=128),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=4)
)

In [103]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)

In [104]:
m2 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="NN_2x128+64_BN__Drop=3_sigmoid_lr=1e-3_bs=512",
        verbose=True
        )

cuda:0


In [105]:
lh = m2(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.6986, validation_loss:0.5281, val_accuracy:0.7891
Epoch [2/100], average_loss:0.5049, validation_loss:0.4605, val_accuracy:0.8047
Epoch [3/100], average_loss:0.4556, validation_loss:0.4726, val_accuracy:0.8047
Epoch [4/100], average_loss:0.4300, validation_loss:0.4081, val_accuracy:0.8516
Epoch [5/100], average_loss:0.4145, validation_loss:0.3831, val_accuracy:0.8438
Epoch [6/100], average_loss:0.4028, validation_loss:0.4670, val_accuracy:0.7344
Epoch [7/100], average_loss:0.3969, validation_loss:0.4447, val_accuracy:0.8672
Epoch [8/100], average_loss:0.3908, validation_loss:0.3643, val_accuracy:0.8516
Epoch [9/100], average_loss:0.3850, validation_loss:0.4736, val_accuracy:0.8359
Epoch [10/100], average_loss:0.3826, validation_loss:0.3974, val_accuracy:0.7500
Epoch [11/100], average_loss:0.3783, validation_loss:0.3844, val_accuracy:0.8203
Epoch [12/100], average_loss:0.3767, validation_loss:0.3663, val_accuracy:0.8125
Epoch [13/100], average_loss:0.3740, 

In [106]:
m2.tb.close()

In [107]:
m2.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m2.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [108]:
x_pred

tensor([1, 0, 1,  ..., 1, 0, 1])

In [109]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8007112220390913

In [110]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8576785714285714

In [111]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[2943,  901,   43,  268],
       [ 526, 9788,   67,  216],
       [  15,   59,  398,   20],
       [ 154,   85,   37, 1280]], dtype=int64)

# without BN

In [112]:
model = nn.Sequential(
            nn.Linear(in_features=29, out_features=64),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=64),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=64),
            #nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=4)
)

In [113]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)

In [114]:
m3 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="NN_3x64_Drop=3_sigmoid_lr=1e-3_bs=512",
        verbose=True
        )

cuda:0


In [115]:
lh = m3(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:1.0389, validation_loss:0.9032, val_accuracy:0.6484
Epoch [2/100], average_loss:0.7663, validation_loss:0.6301, val_accuracy:0.8047
Epoch [3/100], average_loss:0.6120, validation_loss:0.5695, val_accuracy:0.8203
Epoch [4/100], average_loss:0.5515, validation_loss:0.5018, val_accuracy:0.8516
Epoch [5/100], average_loss:0.4937, validation_loss:0.4602, val_accuracy:0.7891
Epoch [6/100], average_loss:0.4595, validation_loss:0.4491, val_accuracy:0.8359
Epoch [7/100], average_loss:0.4463, validation_loss:0.4431, val_accuracy:0.7969
Epoch [8/100], average_loss:0.4398, validation_loss:0.4496, val_accuracy:0.8203
Epoch [9/100], average_loss:0.4332, validation_loss:0.4331, val_accuracy:0.8281
Epoch [10/100], average_loss:0.4294, validation_loss:0.4267, val_accuracy:0.8516
Epoch [11/100], average_loss:0.4257, validation_loss:0.4200, val_accuracy:0.7969
Epoch [12/100], average_loss:0.4187, validation_loss:0.4114, val_accuracy:0.8125
Epoch [13/100], average_loss:0.4102, 

In [123]:
m3.tb.close()

In [116]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [117]:
x_pred

tensor([1, 0, 1,  ..., 1, 0, 1])

In [118]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8164482557063294

In [119]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8720238095238095

In [120]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[3835,  116,   43,  161],
       [1090, 9170,  103,  234],
       [  18,   29,  426,   19],
       [ 224,   56,   57, 1219]], dtype=int64)

In [122]:
#torch.save(m3.model, "models_and_losses/NN_3x64_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-100.pt")
#torch.save(m3.model.state_dict(), "models_and_losses/NN_3x64_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-100_state_dict.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sigmoid. It won't be checked for correctness upon

# Without BN deeper

In [125]:
model = nn.Sequential(
            nn.Linear(in_features=29, out_features=64),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=64),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=64),
            #nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=64),
            #nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=4)
)

In [126]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)

In [127]:
m3 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="NN_4x64_Drop=3_sigmoid_lr=1e-3_bs=512",
        verbose=True
        )

cuda:0


In [128]:
lh = m3(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:1.0039, validation_loss:0.9589, val_accuracy:0.5781
Epoch [2/100], average_loss:0.8350, validation_loss:0.6105, val_accuracy:0.8438
Epoch [3/100], average_loss:0.5985, validation_loss:0.5622, val_accuracy:0.7734
Epoch [4/100], average_loss:0.5230, validation_loss:0.4893, val_accuracy:0.7891
Epoch [5/100], average_loss:0.4724, validation_loss:0.4678, val_accuracy:0.8281
Epoch [6/100], average_loss:0.4577, validation_loss:0.4627, val_accuracy:0.7969
Epoch [7/100], average_loss:0.4476, validation_loss:0.4497, val_accuracy:0.8750
Epoch [8/100], average_loss:0.4415, validation_loss:0.4587, val_accuracy:0.8203
Epoch [9/100], average_loss:0.4388, validation_loss:0.4622, val_accuracy:0.7500
Epoch [10/100], average_loss:0.4360, validation_loss:0.4549, val_accuracy:0.7656
Epoch [11/100], average_loss:0.4312, validation_loss:0.4506, val_accuracy:0.8047
Epoch [12/100], average_loss:0.4293, validation_loss:0.4399, val_accuracy:0.8594
Epoch [13/100], average_loss:0.4267, 

In [129]:
m3.tb.close()

In [130]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [131]:
x_pred

tensor([1, 0, 1,  ..., 1, 0, 1])

In [132]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.811118113027148

In [133]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8693452380952381

In [134]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[3786,  112,   66,  191],
       [1084, 9146,  118,  249],
       [  13,   28,  426,   25],
       [ 214,   54,   41, 1247]], dtype=int64)

# deeper with bottlenecks

In [136]:
model = nn.Sequential(
            nn.Linear(in_features=29, out_features=64),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=128),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=32),
            #nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),#nn.LeakyReLU(),
            nn.Linear(in_features=32, out_features=128),
            #nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=64),
            #nn.BatchNorm1d(num_features=64),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=64, out_features=4)
)

In [137]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)

In [138]:
m3 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="NN_[64-128-32-128-64]_Drop=3_sigmoid_lr=1e-3_bs=512",
        verbose=True
        )

cuda:0


In [139]:
lh = m3(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:1.0063, validation_loss:0.9742, val_accuracy:0.6328
Epoch [2/100], average_loss:0.9015, validation_loss:0.6452, val_accuracy:0.7422
Epoch [3/100], average_loss:0.6241, validation_loss:0.6378, val_accuracy:0.7578
Epoch [4/100], average_loss:0.5889, validation_loss:0.6122, val_accuracy:0.8281
Epoch [5/100], average_loss:0.5732, validation_loss:0.6032, val_accuracy:0.7656
Epoch [6/100], average_loss:0.5571, validation_loss:0.6130, val_accuracy:0.7734
Epoch [7/100], average_loss:0.5430, validation_loss:0.6031, val_accuracy:0.8281
Epoch [8/100], average_loss:0.5250, validation_loss:0.5791, val_accuracy:0.8594
Epoch [9/100], average_loss:0.5001, validation_loss:0.5414, val_accuracy:0.7812
Epoch [10/100], average_loss:0.4757, validation_loss:0.5183, val_accuracy:0.8984
Epoch [11/100], average_loss:0.4622, validation_loss:0.5372, val_accuracy:0.7969
Epoch [12/100], average_loss:0.4564, validation_loss:0.5141, val_accuracy:0.8203
Epoch [13/100], average_loss:0.4514, 

In [145]:
lh = m3(epochs=range(100,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [101/200], average_loss:0.3394, validation_loss:0.3744, val_accuracy:0.8828
Epoch [102/200], average_loss:0.3398, validation_loss:0.3555, val_accuracy:0.8438
Epoch [103/200], average_loss:0.3388, validation_loss:0.3665, val_accuracy:0.8359
Epoch [104/200], average_loss:0.3350, validation_loss:0.3708, val_accuracy:0.8281
Epoch [105/200], average_loss:0.3364, validation_loss:0.3495, val_accuracy:0.8828
Epoch [106/200], average_loss:0.3347, validation_loss:0.3694, val_accuracy:0.8438
Epoch [107/200], average_loss:0.3383, validation_loss:0.3706, val_accuracy:0.8750
Epoch [108/200], average_loss:0.3355, validation_loss:0.3706, val_accuracy:0.8125
Epoch [109/200], average_loss:0.3366, validation_loss:0.3691, val_accuracy:0.8438
Epoch [110/200], average_loss:0.3347, validation_loss:0.3487, val_accuracy:0.8516
Epoch [111/200], average_loss:0.3340, validation_loss:0.3640, val_accuracy:0.8594
Epoch [112/200], average_loss:0.3349, validation_loss:0.3800, val_accuracy:0.8750
Epoch [113/200],

In [154]:
m3.tb.close()

In [146]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [147]:
x_pred

tensor([1, 0, 1,  ..., 1, 0, 1])

In [148]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8175120934534321

In [149]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8744047619047619

In [150]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[3782,  115,   60,  198],
       [1035, 9214,  115,  233],
       [  13,   23,  435,   21],
       [ 195,   50,   52, 1259]], dtype=int64)

In [151]:
#torch.save(m3.model, "models_and_losses/NN_[64-128-32-128-64]_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-200.pt")
#torch.save(m3.model.state_dict(), "models_and_losses/NN_[64-128-32-128-64]_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-200_state_dict.pt")

In [152]:
m3.model

Sequential(
  (0): Linear(in_features=29, out_features=64, bias=True)
  (1): Dropout(p=0.3, inplace=False)
  (2): Sigmoid()
  (3): Linear(in_features=64, out_features=128, bias=True)
  (4): Dropout(p=0.3, inplace=False)
  (5): Sigmoid()
  (6): Linear(in_features=128, out_features=32, bias=True)
  (7): Dropout(p=0.3, inplace=False)
  (8): Sigmoid()
  (9): Linear(in_features=32, out_features=128, bias=True)
  (10): Dropout(p=0.3, inplace=False)
  (11): Sigmoid()
  (12): Linear(in_features=128, out_features=64, bias=True)
  (13): Dropout(p=0.3, inplace=False)
  (14): Sigmoid()
  (15): Linear(in_features=64, out_features=4, bias=True)
)

# simpler model

In [2]:
model = nn.Sequential(
            nn.Linear(in_features=29, out_features=128),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=128),
            #nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.3),
            nn.Sigmoid(),
            nn.Linear(in_features=128, out_features=4)
)

In [3]:
model

Sequential(
  (0): Linear(in_features=29, out_features=128, bias=True)
  (1): Dropout(p=0.3, inplace=False)
  (2): Sigmoid()
  (3): Linear(in_features=128, out_features=128, bias=True)
  (4): Dropout(p=0.3, inplace=False)
  (5): Sigmoid()
  (6): Linear(in_features=128, out_features=4, bias=True)
)

In [155]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)

In [156]:
m3 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="NN_2x128_Drop=3_sigmoid_lr=1e-3_bs=512",
        verbose=True
        )

cuda:0


In [157]:
lh = m3(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/100], average_loss:0.7731, validation_loss:0.4978, val_accuracy:0.8594
Epoch [2/100], average_loss:0.4757, validation_loss:0.4431, val_accuracy:0.8125
Epoch [3/100], average_loss:0.4292, validation_loss:0.4039, val_accuracy:0.7734
Epoch [4/100], average_loss:0.3968, validation_loss:0.3898, val_accuracy:0.8125
Epoch [5/100], average_loss:0.3759, validation_loss:0.3695, val_accuracy:0.8203
Epoch [6/100], average_loss:0.3643, validation_loss:0.3647, val_accuracy:0.8594
Epoch [7/100], average_loss:0.3573, validation_loss:0.3661, val_accuracy:0.8984
Epoch [8/100], average_loss:0.3541, validation_loss:0.3587, val_accuracy:0.8125
Epoch [9/100], average_loss:0.3499, validation_loss:0.3548, val_accuracy:0.7891
Epoch [10/100], average_loss:0.3482, validation_loss:0.3588, val_accuracy:0.8516
Epoch [11/100], average_loss:0.3462, validation_loss:0.3566, val_accuracy:0.8594
Epoch [12/100], average_loss:0.3442, validation_loss:0.3576, val_accuracy:0.8281
Epoch [13/100], average_loss:0.3430, 

In [158]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [159]:
x_pred

tensor([1, 0, 1,  ..., 1, 0, 1])

In [160]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8203182056679881

In [161]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8757738095238096

In [162]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[3788,  138,   47,  182],
       [ 998, 9237,  113,  249],
       [  17,   23,  434,   18],
       [ 198,   52,   52, 1254]], dtype=int64)

In [164]:
#torch.save(m3.model, "models_and_losses/NN_2x128_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-100.pt")
#torch.save(m3.model.state_dict(), "models_and_losses/NN_2x128_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-100_state_dict.pt")

In [165]:
lh = m3(epochs=range(100,200), train_loader=train_loader, validation_loader=valid_loader)

Epoch [101/200], average_loss:0.3059, validation_loss:0.3298, val_accuracy:0.8906
Epoch [102/200], average_loss:0.3056, validation_loss:0.3184, val_accuracy:0.8984
Epoch [103/200], average_loss:0.3046, validation_loss:0.3198, val_accuracy:0.8750
Epoch [104/200], average_loss:0.3049, validation_loss:0.3298, val_accuracy:0.8828
Epoch [105/200], average_loss:0.3054, validation_loss:0.3425, val_accuracy:0.8359
Epoch [106/200], average_loss:0.3040, validation_loss:0.3235, val_accuracy:0.8516
Epoch [107/200], average_loss:0.3051, validation_loss:0.3172, val_accuracy:0.9141
Epoch [108/200], average_loss:0.3031, validation_loss:0.3287, val_accuracy:0.8438
Epoch [109/200], average_loss:0.3033, validation_loss:0.3312, val_accuracy:0.8828
Epoch [110/200], average_loss:0.3047, validation_loss:0.3297, val_accuracy:0.9141
Epoch [111/200], average_loss:0.3022, validation_loss:0.3296, val_accuracy:0.9062
Epoch [112/200], average_loss:0.3039, validation_loss:0.3338, val_accuracy:0.8594
Epoch [113/200],

In [166]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [167]:
x_pred

tensor([1, 0, 1,  ..., 1, 0, 1])

In [168]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8284568712860796

In [169]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8805952380952381

In [170]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[3790,  138,   47,  180],
       [ 996, 9332,   82,  187],
       [  17,   34,  419,   22],
       [ 193,   71,   39, 1253]], dtype=int64)

In [171]:
#torch.save(m3.model, "models_and_losses/NN_2x128_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-200.pt")
#torch.save(m3.model.state_dict(), "models_and_losses/NN_2x128_Drop=3_sigmoid_lr=1e-3_bs=512_epoch-200_state_dict.pt")

In [172]:
lh = m3(epochs=range(200,300), train_loader=train_loader, validation_loader=valid_loader)

Epoch [201/300], average_loss:0.2927, validation_loss:0.3124, val_accuracy:0.8750
Epoch [202/300], average_loss:0.2926, validation_loss:0.3193, val_accuracy:0.7891
Epoch [203/300], average_loss:0.2934, validation_loss:0.3131, val_accuracy:0.8750
Epoch [204/300], average_loss:0.2932, validation_loss:0.3192, val_accuracy:0.9062
Epoch [205/300], average_loss:0.2924, validation_loss:0.3201, val_accuracy:0.8750
Epoch [206/300], average_loss:0.2912, validation_loss:0.3376, val_accuracy:0.8672
Epoch [207/300], average_loss:0.2922, validation_loss:0.3118, val_accuracy:0.8359
Epoch [208/300], average_loss:0.2929, validation_loss:0.3099, val_accuracy:0.9297
Epoch [209/300], average_loss:0.2928, validation_loss:0.3159, val_accuracy:0.8984
Epoch [210/300], average_loss:0.2930, validation_loss:0.3276, val_accuracy:0.8672
Epoch [211/300], average_loss:0.2932, validation_loss:0.3162, val_accuracy:0.8828
Epoch [212/300], average_loss:0.2929, validation_loss:0.3161, val_accuracy:0.8203
Epoch [213/300],

In [166]:
m3.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float().cuda()).cpu().detach(), axis=1)

In [ ]:
x_pred

In [ ]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

In [ ]:
accuracy_score(y_true=y_test, y_pred=x_pred)

In [ ]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1